## Imports

In [ ]:
import logging
from datetime import datetime as DT
from datetime import timedelta as TD
from dateutil import parser

from rich import print
import pandas as pd
from pandas import Series
from pandas import DataFrame as DF
import plotly.express as px
from core.s3_utils import S3_Bucket
from core.constants import *
from core.time_series_processing import preprocess_date

## Setup

Lets extract some raw time series... 

In [ ]:
VIN = "VF1AG000166417017"

bucket = S3_Bucket()
def get_renault_raw_ts(vin:str) -> DF:
    return (
        bucket.read_parquet_df(f"raw_ts/renault/time_series/{vin}.parquet")
        .set_index("date", drop=False)
        .sort_index()
        .assign(soc=lambda raw_ts:raw_ts["charging.battery_level"] * 100)
    )
raw_tss:DF = get_renault_raw_ts(VIN)

## Time series visualization
Let's look for variables that we could use to estimate the soh.

In [ ]:
# with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
#     display(raw_tss["VF1AG000065981036"])

In [ ]:
COLS_TO_DISPALY = [
    # "charging.battery_charge_type",
    "charging.distance_to_complete_charge",
    # "charging.plugged_in",
    # "charging.status",
    # "diagnostics.odometer",
    "charging.battery_energy",
    "soc",
    "charging.estimated_range"
]
# px.line(raw_ts, COLS_TO_DISPALY)
px.line(raw_tss[COLS_TO_DISPALY], )

# Soh based on battery_energy

In [ ]:
soh = raw_tss["charging.battery_energy"] / raw_tss["soc"]
px.line(soh)